## construct graph from SVIs

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from transformers import SegformerFeatureExtractor, SegformerForSemanticSegmentation
import numpy as np
import networkx as nx
from PIL import Image
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
from torch_geometric.nn import GCNConv

import requests
import torch
from PIL import Image
from transformers import AutoImageProcessor, Mask2FormerForUniversalSegmentation


# load Mask2Former fine-tuned on Cityscapes semantic segmentation
# feature_extractor = AutoImageProcessor.from_pretrained("facebook/mask2former-swin-large-cityscapes-semantic")
# model = Mask2FormerForUniversalSegmentation.from_pretrained("facebook/mask2former-swin-large-cityscapes-semantic").cuda()

# # 加载模型
feature_extractor = SegformerFeatureExtractor.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512")
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b0-finetuned-ade-512-512").cuda()
model.eval()

# Get list of class names as column names
class_names = list(model.config.id2label.values())[1:]

# 定义一个函数，用来计算bounding box
def get_bounding_box(nodes):
    min_i, min_j = nodes.min(axis=0)
    max_i, max_j = nodes.max(axis=0)
    return (min_j, min_i, max_j+1, max_i+1)

def get_mask(image):
    inputs = feature_extractor(image, return_tensors="pt").to(model.device)
    # 对输入图片进行分割
    with torch.no_grad():    
        outputs = model(**inputs)
        logits = outputs.logits  # shape (batch_size, num_labels, height/4, width/4)
        predictions = torch.argmax(logits, dim=1).cpu().numpy().squeeze().astype('uint8')  # shape (batch_size, height/4, width/4)
    return predictions

def get_graph(predictions):
    
    mask = predictions
    labels = list(np.unique(predictions))

    # 创建空的图
    G = nx.Graph()
    
    # 添加节点
    for label in labels:
        nodes = np.argwhere(mask == label)  # 获取当前标签对应的像素点坐标
        centroid = np.mean(nodes, axis=0)   # 计算节点的中心坐标
        count = len(nodes)                  # 获取节点的像素点数目
        total_count = mask.size             # 获取整张图的像素点数目
        proportion = count / total_count*100   # 计算节点所占比例
        G.add_node(label, centroid=centroid, label=class_names[label-1], proportion=proportion)  # 添加节点，并记录节点的中心坐标和节点所占比例
    
    # 添加边
    for i, (label_i, data_i) in enumerate(G.nodes(data=True)):
        for j, (label_j, data_j) in enumerate(G.nodes(data=True)):
            if i >= j:  # 避免重复添加边
                continue
            dist = np.linalg.norm(data_i['centroid'] - data_j['centroid'])  # 计算节点之间的欧式距离
            if dist < 42:  # 如果两个节点之间的距离小于一个阈值，则添加一条边
                G.add_edge(label_i, label_j)

    # 将节点的比例作为特征嵌入图中
    node_features = np.array([data['proportion'] for _, data in G.nodes(data=True)])

    return node_features,G

def get_embedding(node_features, G):
    
    # 定义嵌入层将特征从 1 维映射到 5 维
    x = torch.tensor(node_features, dtype=torch.float)
    x = nn.Embedding(len(x), 5)(torch.arange(len(x)).long())  # 15 表示节点数，5 表示嵌入后的特征维度

    edge_index = from_networkx(G).edge_index

    data = Data(x=x, edge_index=edge_index,)

    # 定义 GCN 模型
    class GCN(torch.nn.Module):
        def __init__(self, in_channels, hidden_channels, out_channels):
            super(GCN, self).__init__()
            self.conv1 = GCNConv(in_channels, hidden_channels)
            self.conv2 = GCNConv(hidden_channels, out_channels)

        def forward(self, x, edge_index):
            x = F.relu(self.conv1(x, edge_index))
            x = self.conv2(x, edge_index)
            x = F.dropout(x, training=self.training)
            x = torch.mean(x, dim=0)  # 将所有节点的特征求平均得到图的嵌入向量
            x = torch.nn.Linear(x.shape[0], 5)(x)  # 将嵌入向量映射为 5 维向量
            return x

    # 初始化 GCN 模型并传入数据进行计算
    model = GCN(in_channels=5, hidden_channels=16, out_channels=5)
    embedding = model(data.x, data.edge_index)

    return embedding

In [ ]:
import os
import csv
from tqdm import tqdm

# 定义要处理的文件夹路径
folder_path = "E:/Dataset/GNN_Perception/wuhan_badu_SVI/baidu2023_pinjie/"

# 定义输出CSV文件路径
output_file_path = "E:/Dataset/GNN_Perception/wuhan_badu_SVI/2023/embedding.csv"

# 遍历文件夹中的所有文件
for filename in tqdm(os.listdir(folder_path)):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # 读取图像
        image = Image.open(os.path.join(folder_path, filename))
        (node_features,G) = get_graph(get_mask(image))
        embedding = get_embedding(node_features,G)

        # 将处理后的数据存储到CSV文件中
        with open(output_file_path, mode='a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            row = [filename, embedding]
            writer.writerow(row)

## big graph

In [ ]:
##获取同一类别街景的边，构建实体大图

import csv
import os
from tqdm import tqdm

def get_edge(predictions):
    
    labels = list(np.unique(predictions))

    # 创建空的列表存储节点坐标
    nodes = []

    # 添加节点
    for label in labels:
        indices = np.argwhere(predictions == label)  # 获取当前标签对应的像素点坐标
        centroid = np.mean(indices, axis=0)   # 计算节点的中心坐标
        nodes.append((label-1, centroid))
    
    e= []
    # 添加边
    for i, (label_i, centroid_i) in enumerate(nodes):
        for j, (label_j, centroid_j) in enumerate(nodes):
            if i >= j:  # 避免重复添加边
                continue
            dist = np.linalg.norm(centroid_i - centroid_j)  # 计算节点之间的欧式距离
            
            if dist < 42:  # 如果两个节点之间的距离小于一个阈值，则添加一条边
                e.append([label_i, label_j])
    return e

In [ ]:
import os
import csv
from tqdm import tqdm

# 读取CSV文件并获取所有id值
with open('E:/Dataset/GNN_Perception/fig_out/pre2.csv', 'r') as f:
    reader = csv.reader(f)
    id_list = [row[1] for row in reader]
    
new_id_list = [x.split('_')[0] for x in id_list]
new_id_list

# 定义要处理的文件夹路径
folder_path = "E:/Dataset/GNN_Perception/wuhan_badu_SVI/baidu2023_pinjie"
# folder_path = "E:/Dataset/GNN_Perception/wuhan_badu_SVI/baidu2022"

data=[]
# 遍历文件夹中的所有图片
for filename in tqdm(os.listdir(folder_path)):
    # 解析图片名称以获取其id值
    file_id = filename.split('_')[1]
    # 如果该id在csv文件中，则将该图片复制到新的文件夹中
    if file_id in new_id_list:
        # 读取图像
        image = Image.open(os.path.join(folder_path, filename))
        data.append(get_edge(get_mask(image)))

headers = ['src', 'dst']  # CSV文件头部

with open('E:/Dataset/GNN_Perception/fig_out/road_grapg_pre22.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['src', 'dst'])
    for sublist in data:
        for row in sublist:
            writer.writerow(row)

In [ ]:
import torch
from torch_geometric.data import Data
import pandas as pd
from torch_geometric.utils import to_networkx
import networkx as nx
import matplotlib.pyplot as plt

edge_data = pd.read_csv('E:/Dataset/GNN_Perception/fig_out/road_grapg_pre33.csv')
nodes = list(set(list(edge_data['src']) + list(edge_data['dst'])))
edge_index = torch.tensor([edge_data['src'], edge_data['dst']], dtype=torch.long)

data = Data(edge_index=edge_index, num_nodes=0)
G = to_networkx(data, to_undirected=True)
fig, ax = plt.subplots(figsize=(8,6.5))

degree_centrality = nx.centrality.degree_centrality(G)  # save results in a variable to use again

pos = nx.spring_layout(G, iterations=300, seed=1235, k=8)
# pos = nx.circular_layout(G)

node_color = [degree_centrality[node] for node in G.nodes()]

# Get list of class names as column names
class_names = list(model.config.id2label.values())[1:]
# 创建字典，key 为 class 表中的元素，value 为该元素在列表中的索引
class_dict = {class_names[i]: i for i in range(len(class_names))}

# 存储节点编号和它们对应的类别索引
labels = {}
for node in nodes:
    # 如果节点对应的类别不存在于 class_dict 中，则跳过该节点
    if node >= len(class_names):
        continue
    # 根据 node 在 class_dict 中查找对应的 value
    node_value = class_dict[class_names[node]]
    labels[node] = node_value
# 根据 labels 中的索引值获取相应的类别名称，并创建新的字典
class_labels = {k: class_names[v] for k, v in labels.items()}

nx.draw(G, node_size=80, pos=pos, with_labels=True, labels=class_labels, font_size=7, node_color=node_color, cmap=plt.cm.Purples, vmax=1.0, edge_color='gray', alpha=0.9,width=0.2)

# Add legend for node color
sm = plt.cm.ScalarMappable(cmap=plt.cm.Purples, norm=plt.Normalize(vmin=min(degree_centrality.values()), vmax=max(degree_centrality.values())))
sm._A = []
cb = plt.colorbar(sm,shrink=0.4)
cb.set_label('Degree Centrality', fontsize=14, labelpad=10)

# plt.savefig('E:/Dataset/GNN_Perception/fig_out/pre-2.svg')
plt.show()

# 计算节点的度
degrees = dict(G.degree())
sorted_degrees_desc = sorted(degrees.items(), key=lambda x: x[1], reverse=True)
# 打印结果
print(sorted_degrees_desc)

In [ ]:
centrality = sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)

In [ ]:
import matplotlib.pyplot as plt

mask = predictions
# 获取唯一的标签列表
labels = list(np.unique(mask))
# labels.remove(0)  # 去除背景标签0

# 创建空的图
G = nx.Graph()

# 添加节点
for label in labels:
    nodes = np.argwhere(mask == label)  # 获取当前标签对应的像素点坐标
    centroid = np.mean(nodes, axis=0)   # 计算节点的中心坐标
    count = len(nodes)                  # 获取节点的像素点数目
    total_count = mask.size             # 获取整张图的像素点数目
    proportion = count / total_count*100   # 计算节点所占比例
    G.add_node(label, centroid=centroid, label=class_names[label-1], proportion=proportion)  # 添加节点，并记录节点的中心坐标和节点所占比例

# 添加边
for i, (label_i, data_i) in enumerate(G.nodes(data=True)):
    for j, (label_j, data_j) in enumerate(G.nodes(data=True)):
        if i >= j:  # 避免重复添加边
            continue
        dist = np.linalg.norm(data_i['centroid'] - data_j['centroid'])  # 计算节点之间的欧式距离
        if dist < 42:  # 如果两个节点之间的距离小于一个阈值，则添加一条边
            G.add_edge(label_i, label_j)

# 将节点的比例作为特征嵌入图中
node_features = np.array([data['proportion'] for _, data in G.nodes(data=True)])
nx.set_node_attributes(G, dict(enumerate(node_features)), 'proportion')

# # 获取整体特征，768维度向量
# vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224')
# features = vit_model(inputs.pixel_values).last_hidden_state.mean(dim=1).squeeze()

# 打印图的基本信息
print(f'Number of nodes: {G.number_of_nodes()}')
print(f'Number of edges: {G.number_of_edges()}')
print(f'Nodes: {G.nodes(data=True)}')

fig, ax = plt.subplots(figsize=(10,5))
pos = nx.spring_layout(G, iterations=50, seed=12345)

# 在绘制图形时，将特征作为参数传递给nx.draw()函数
# nx.draw(G, node_size=50, pos={k: v['centroid'] for k, v in G.nodes(data=True)}, with_labels=True, labels=nx.get_node_attributes(G,'label'))  
nx.draw(G, node_size=500, pos=pos, with_labels=True, labels=nx.get_node_attributes(G,'label'), 
        node_color=labels, cmap=plt.cm.Blues)  

# plt.savefig('E:/Dataset/GNN_Perception/fig_out/grapg.svg',format='svg',bbox_inches='tight')
